RESOURCES

https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/

https://python.langchain.com/docs/tutorials/rag/

https://scalexi.medium.com/implementing-a-retrieval-augmented-generation-rag-system-with-openais-api-using-langchain-ab39b60b4d9f


In [1]:
## Imports
import os
from dotenv import load_dotenv

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

Generate and save [Langchain API key](https://docs.smith.langchain.com/how_to_guides/setup/create_account_api_key) to `.env`. 

[LangSmith](https://www.langchain.com/langsmith) is a helpful monitoring tool developed by LangChain to trace the behavior of your application. This is particularly helpful when your application has RAG and history. 

In [2]:
## Setup the API keys
load_dotenv()

## If you want LangSmith to trace your runs, set this environmental variable
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

To provide context-rich data to the interior designer, I created a text repository comprising of fake expert data.
I used ChatGPT to create the fake data.
For easy handling, separate text files were created for different areas of the house.

In [3]:
## Function for reading text files containing information for RAG
def read_txt_files_in_folder(folder_path):
    all_texts = []
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    filtered_content = ''.join([char for char in content if char not in ['**','#','##','###']])
                    all_texts.append(filtered_content)
    
    return all_texts

In [4]:
## Reading the text files
text = read_txt_files_in_folder('data/')

In [5]:
# print(len(text))
# print(len(text[0])) 

The text data should be split into manageable chunks that fit within the context window of the model.
`RecursiveCharacterTextSplitter` recursively splits the text data into fragments using characters from the default list `["\n\n", "\n", " ", ""]`, by finding the one that works. Chunks are created so that they are less than or equal in length to `chunk_size`.
While there are several other text splitters, [this splitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/) works best for generic text. 

`create_documents` is a little confusing since we have already processed the text documents into an array of strings. This method takes the array of string data as input and returns a set of 'document' objects that contain the split chunks. 

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200, 
    add_start_index=True
)

## Converting text data into documents
docs = text_splitter.create_documents(text)

In [7]:
# print(len(docs))
# print(docs[1])

Now we have domain specific information, loaded from text files and processed into a format suitable for LangChain. For each user query, we should retrieve the appropriate snippets and provide them as context to the model. The RAG process is only as good as the retrieved snippets' relevance and quality. LangChain has [implementations](https://python.langchain.com/v0.2/docs/concepts/#retrieval) of multiple retrieval techniques that are suitable for different usecases.  

Here, I use vector stores, one of the the simplest methods of retrieval. This is a beginner friendly method. Specifically, [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma/) vector database was used to prepare the vector store. Here, unstructured text data is transformed into embeddings and during query phase, the query is converted to an embedding, the appropriate snippets are retrieved based on embedding similarity and an index corresponding to the relevant chunk is returned. Embeddings are computed using [OpenAI embedding models](https://python.langchain.com/docs/integrations/text_embedding/openai/).

It is important to note that, in addition to the retrieval method, the size of the chunks and overlap used during text splitting play a key role on the effectiveness of RAG inputs.

In [8]:
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

The output of an LLM will be only as good as the prompt we give. [LangChain Hub](https://smith.langchain.com/hub) consists of pre-defined prompts for diverse usecases. created a prompt template based on `rag-prompt` from the Hub.   
[PromptTemplate](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/) converts the string prompt to a LangChain prompt template. 

In [9]:
## Template based on hub.pull("rlm/rag-prompt")
template = """Use the following pieces of context to answer the questions related to interior and exterior design of homes. Please respond without using double-quotation marks. 
If the question is not related to interior or exterior design, politely say that your are an assistant helping with interior and exterior design and tell the user to ask relavant questions, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [10]:
# print(custom_rag_prompt)

As the LLM, I use OpenAI's `gpt-4o` model through `ChatOpenAI` API of LangChain. To try other OpenAI models, you can simply update the `model_name` argument with a different model. Check [OpenAI Plaform](https://platform.openai.com/docs/models) for all available models. 

In [11]:
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o")

We have all the components required query our AI interior designer model. Now we create a chain that composes all the components and functions together. We use `RunnablePassthrough` to pass the user query into the prompt.     

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What color scheme should I use in a model kitchen?") 

In [ ]:
## Out of context question
rag_chain.invoke("What is the capital of the United States?") 

Adding history to the chat
Here, I also experiment with LangChain's buil-in chain constructors to create chains.
[Reference](https://python.langchain.com/docs/tutorials/qa_chat_history/) 

https://python.langchain.com/v0.1/docs/modules/chains/

In [ ]:
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.messages import AIMessage, HumanMessage

# RAG with buil-in chain constructors and without history

Note that here I am using ChatPromptTemplate for creating the prompt. This is suitable for prompting chat models (that is, we have back-and-forth). We can input a list of chat messages and assign roles to the messages. [Learn more about prompt templates](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/)

`create_stuff_documents_chain` create a chain for passing a list of Documents to a model. [link](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html)

In [ ]:
## Using ChatPromptTemplate
system_prompt = ("Use the following pieces of context to answer the questions related to interior and exterior design of homes." 
                 "Please respond without using double-quotation marks." 
                 "If the question is not related to interior or exterior design, politely say that you don't know and tell the user to ask relavant questions," 
                 "don't try to make up an answer."
                 "Use three sentences maximum and keep the answer as concise as possible."
                 "\n\n"
                 "{context}")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}")
    ]
)

## This chain passes a list of documents to the LLM
qna_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, qna_chain)

response = rag_chain.invoke({"input": "What color scheme should I use in a bathroom?"})
print(response["answer"])

response = rag_chain.invoke({"input": "What color dress should I wear to a party?"})
print(response["answer"])

# Adding chat history

This part of the notebook closely follows this [tutorial](https://python.langchain.com/docs/tutorials/qa_chat_history/) from LangChain.
 
When incorporating history, the retriever should retrieve documents based on current context and history. So, use another LLM call to create a prompt that incorporates current query and chat history to create the retriever.

We must rephrase the input query to incorporate historical messages. We use a sub-chain to create a ['history aware'](https://python.langchain.com/docs/tutorials/qa_chat_history/#adding-chat-history) retriever which we then use with the LLM to get the response.

[create_history_aware_retriever](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.history_aware_retriever.create_history_aware_retriever.html) creates a chain that takes input and conversation history and returns documents.  

In [16]:
## Creating a prompt by combining chat history and user query

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)


We generate the `question_answer_chain` again and build the final rag chain to apply the `history_aware_retriever` and `question_answer_chain` in sequence.

In [17]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

qna_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qna_chain)

Before invoking the chain, we must manage `"chat_history"`. This is maintained as a list.   

In [ ]:
chat_history = []

question = "What kind of design theme should I use for my living room?"
ai_msg1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg1["answer"]),
    ]
)

print(ai_msg1["answer"])
print()

question2 = "What kind of furniture should I use there?"

## Checking the intermediate output of the rephrasing prompt step
out_chain = (
    contextualize_q_prompt
    | llm
    | StrOutputParser()
)

temp_out = out_chain.invoke({"input": question2, "chat_history": chat_history})

print(temp_out)
print()

ai_msg2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})

print(ai_msg2["answer"])

We developed the logic for retaining chat history between messages. However, this is still very manual. We can wrap our chat model in a [LangGraph](https://langchain-ai.github.io/langgraph/) application to automatically persist the message history.   

[Persistence](https://langchain-ai.github.io/langgraph/concepts/persistence/) layers are crucial when maintaining memory within chat applications. LangGraph has is own built-in persistence layer.  

We should install LangGraph separately.

`pip install -U langgraph`

In [19]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

The dict `State` represents the state of the application. It has the same input and output keys as `rag_chain`.

In [20]:
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str

Next we define the function (or node) that calls the model, in other words, runs the `rag_chain`.
This function updates the graph state by updating the chat history with the input message and response.

In [21]:
def call_model(state: State):
    response = rag_chain.invoke(state)

    ## Invoking rag_chain outputs a model response and updates the state. So the function 'returns' 
    ## the new graph state.
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context":response["context"],
        "answer": response["answer"],
    }

Now let's create the graph and compile it with a checkpointer object.
We can choose where we would like to persist the state. In this case, we choose the memory.

In [22]:
workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

## Compiling
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

A chat application should handle interactions with multiple users.

This application supports multiple conversations through multiple threads, where each thread has a unique indentifier.

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

result = app.invoke(
    {"input": "What material should I use for kitchen cabinets?"},
    config = config,
)

print(result["answer"])

In [ ]:
result = app.invoke(
    {"input": "What are reclaimed woods?"},
    config = config,
)

print(result["answer"])

In [ ]:
result = app.invoke(
    {"input": "Why are they trendy?"},
    config = config,
)

print(result["answer"])

See what happens when I use a different thread ID.

In [ ]:
config2 = {"configurable": {"thread_id": "abc456"}}

## Eventhough this has a different thread ID, sometimes we get an answer because RAG appends relevant context if there are relevant words in the questions.
result = app.invoke(
    {"input": "Why are they trendy?"},
    config = config2,
)

print(result["answer"])

We can look at the chat history using `get_state` as shown below. 

The tutorial has an [illustration](https://python.langchain.com/docs/tutorials/qa_chat_history/#tying-it-together) that ties everything together and gives you the big picture of the application.

In [ ]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()